In [1]:
# Importação de bibliotecas
import pandas as pd

In [2]:
# Variáveis utilizadas
fonte = 'is_agro'
dataset_name = 'area_agropecuaria_pastagem'
raiz = '/home/eduardo/repositorios/IS_Agro/databases'

### Ouro

In [3]:
# Tabela do LAPIG para pastagem estadual
df_pastagem_uf = (
    pd.read_parquet(f'{raiz}/prata_lapig_pastagem_uf.parquet')
    .reset_index()
    .convert_dtypes()
)
df_pastagem_br = (
    df_pastagem_uf
    .groupby('data')
    .agg({'area': 'sum'})
    .reset_index()
)

In [4]:
# Correção dos dados de pastagem a partir dos cálculos da Embrapa Meio Ambiente
df_pastagem_rev = (
    df_pastagem_uf
    .merge(
        df_pastagem_br,
        how='left',
        on='data',
        suffixes=['_uf', '_br']
    )
    .merge(
        pd.read_parquet(f'{raiz}/prata_{fonte}_usodaterra_agro_br.parquet') # Tabela gerada pela equipe da Embrapa Meio Ambiente, Campinas
        .query('cod_mapbiomas in (15, 12) & data >= 2000')
        .groupby('data')
        .agg({'valor':'sum'}),
        how='left',
        on='data'
    )
    .assign(valor = lambda n: n.area_uf * n.valor/n.area_br)
    .filter(['geocodigo', 'data', 'valor'])
    .convert_dtypes()
)

#df_pastagem_rev.to_parquet(f'{raiz}/ouro_{fonte}_{dataset_name}.parquet')
df_pastagem_rev.to_csv(f'{raiz}/ouro_{fonte}_{dataset_name}.csv')